In [203]:
import pandas as PD
import numpy as NP
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
from selenium.common.exceptions import TimeoutException  
# import ActionChains mouse operation  
from selenium.webdriver.common.action_chains import ActionChains 

from selenium.webdriver.chrome.options import Options
import time
from lxml import etree


# Dynamic get html

In [204]:
def get_html(url):
    try:
        chrome_options=Options()
        chrome_options.add_argument('--headless')
        driver= webdriver.Chrome(chrome_options=chrome_options)  
        driver.get(url)
        temp_height=0
 
        while True:
            #循环将滚动条下拉
            driver.execute_script("window.scrollBy(0,100000)")
            #sleep一下让滚动条反应一下
            time.sleep(2)
            #获取当前滚动条距离顶部的距离
            check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
            #如果两者相等说明到底了
            if check_height==temp_height:
                break
            temp_height=check_height
            print('*****Processing temp_height = {temp_height}*****'.format(temp_height=temp_height))

        page = driver.page_source
        source = page.encode("utf8")
        soup_list = BeautifulSoup(source, 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")
        return soup_list
    except:
        return "ERROR"

In [205]:
soup_list = get_html("https://careers.fenbushicapital.vc/jobs")

/tmp/ipykernel_16036/50791386.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1378*****
*****Processing temp_height = 4570*****
*****Processing temp_height = 7947*****
*****Processing temp_height = 11180*****
*****Processing temp_height = 14277*****
*****Processing temp_height = 17264*****
*****Processing temp_height = 20287*****
*****Processing temp_height = 23516*****
*****Processing temp_height = 26690*****
*****Processing temp_height = 29927*****
*****Processing temp_height = 33100*****
*****Processing temp_height = 36235*****
*****Processing temp_height = 39468*****
*****Processing temp_height = 42821*****
*****Processing temp_height = 45910*****
*****Processing temp_height = 49091*****
*****Processing temp_height = 52317*****
*****Processing temp_height = 55571*****
*****Processing temp_height = 58881*****
*****Processing temp_height = 62069*****
*****Processing temp_height = 65361*****
*****Processing temp_height = 68486*****
*****Processing temp_height = 71736*****
*****Processing temp_height = 74899*****
*****Processing tem

In [206]:
# soup_list

In [214]:
# get the initializd infomation
tag_list = []

job_title_list = []
type_list = {}
industry_list = {}
contract_list = {}
location_list = {}
equity_list = {}
description_list = {}
salary_list = {}
label_list = {}
achievements_list = {}
# loop = 0

for tag in soup_list.find_all('div',class_="job-list-job"):
        # print(tag)
        raw_data = tag.find('a', target = "_blank")
        job_title_list.append(raw_data.text)
        type_list[raw_data.text] = 'Full-time'
        industry_list[raw_data.text] = ['Others']
        contract_list[raw_data.text] = 'talentre'
        # location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-badge-text").text if (not tag.find('span', class_ = "job-list-badge-text").text[0:6] == "Posted") else "Not Posted"
        location_list[raw_data.text] = tag.find('span', class_ = "job-list-company-meta-item").text
        equity_list[raw_data.text] = "Not Posted"
        description_list[raw_data.text] = raw_data['href']
        salary_list[raw_data.text] = "Not Posted"
        label_list[raw_data.text] = []
        achievements_list[raw_data.text] = 'Null'
        for tag_ in tag.find_all('span', class_ = 'job-list-job-skill job-list-job-skill-clickable'):
                label_list[raw_data.text].append(tag_.text)
        # loop += 1
        # if(loop == 10):
        #     break
        # for tag_ in tag.find_all('span',attrs={ 'data-toggle': 'tooltip'}):
        #     if('$' in tag_.text):
        #         tmp = tag_.text.split('$')[1].replace(',','').replace(')','')

In [215]:
# adjust the type_list
soup_list_internship = get_html("https://careers.fenbushicapital.vc/jobs?internshipOnly=true")
for tag in soup_list_internship.find_all('div',class_="job-list-job"):
    type_list[tag.find('a', target = "_blank").text] = 'Internship'

/tmp/ipykernel_16036/50791386.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1378*****
*****Processing temp_height = 4541*****
*****Processing temp_height = 6308*****


In [216]:
# adjust the type_list
url_list = {"https://careers.fenbushicapital.vc/jobs?jobTypes=Engineer":"Engineering",
            "https://careers.fenbushicapital.vc/jobs?markets=FinTech": "Finance",
            "https://careers.fenbushicapital.vc/jobs?markets=Financial+Services": "Finance",
            "https://careers.fenbushicapital.vc/jobs?jobTypes=Marketing": "Marketing"
            }

In [217]:
for url in url_list.keys():
    print("********************Processing {url} for label {label}********************".format(url = url, label = url_list[url]))
    soup_list_type = get_html(url)
    for tag in soup_list_type.find_all('div',class_="job-list-job"):
        raw_data = tag.find('a', target = "_blank")
        if(raw_data.text in industry_list.keys()):
            if (industry_list[raw_data.text] == ['Others']):
                industry_list[raw_data.text] = [url_list[url]]
            elif (not url_list[url] in industry_list[raw_data.text]):
                industry_list[raw_data.text].append(url_list[url])
            else: 
                continue
        else:
            continue


********************Processing https://careers.fenbushicapital.vc/jobs?jobTypes=Engineer for label Engineering********************


/tmp/ipykernel_16036/50791386.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1378*****
*****Processing temp_height = 4572*****
*****Processing temp_height = 7820*****
*****Processing temp_height = 11070*****
*****Processing temp_height = 14212*****
*****Processing temp_height = 17382*****
*****Processing temp_height = 20647*****
*****Processing temp_height = 23922*****
*****Processing temp_height = 27211*****
*****Processing temp_height = 30378*****
*****Processing temp_height = 33520*****
*****Processing temp_height = 36693*****
*****Processing temp_height = 39919*****
*****Processing temp_height = 43016*****
*****Processing temp_height = 46239*****
*****Processing temp_height = 49430*****
*****Processing temp_height = 52554*****
*****Processing temp_height = 55836*****
*****Processing temp_height = 58919*****
*****Processing temp_height = 62026*****
*****Processing temp_height = 65179*****
*****Processing temp_height = 68391*****
*****Processing temp_height = 71599*****
*****Processing temp_height = 74776*****
*****Processing tem

/tmp/ipykernel_16036/50791386.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1289*****
********************Processing https://careers.fenbushicapital.vc/jobs?markets=Financial+Services for label Finance********************


/tmp/ipykernel_16036/50791386.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1378*****
*****Processing temp_height = 4607*****
*****Processing temp_height = 7655*****
*****Processing temp_height = 10647*****
*****Processing temp_height = 13357*****
*****Processing temp_height = 15527*****
********************Processing https://careers.fenbushicapital.vc/jobs?jobTypes=Marketing for label Marketing********************


/tmp/ipykernel_16036/50791386.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1378*****
*****Processing temp_height = 4523*****
*****Processing temp_height = 7760*****
*****Processing temp_height = 10990*****
*****Processing temp_height = 14286*****
*****Processing temp_height = 16759*****


In [222]:
# show all the results
count = 0
total_columns = ['Job title', 'Type of position', 'Job Location', 'Job description', 'Working type', 'Contact', 'Achievements', 'Base salary', 'Equity']
total_values = [[] for _ in total_columns]

for single_job in job_title_list:
    count += 1
    print("*****Processing job {job}*****".format(job=single_job))
    print("Type of position: " + type_list[single_job])
    print("Job location: " + location_list[single_job])
    print("Job description: " + description_list[single_job])
    print("Working type: " + (",".join(str(x) for x in industry_list[single_job])))
    print("Contract: " + contract_list[single_job])
    print("Achievements: " + achievements_list[single_job])
    print("Base salary: " + salary_list[single_job])
    print("Equity: " + equity_list[single_job])
    
    print("label: " + (",".join(str(x) for x in label_list[single_job])))
    
    single_values = [
        single_job,
        type_list[single_job],
        location_list[single_job],
        description_list[single_job],
        ",".join(str(x) for x in industry_list[single_job]),
        contract_list[single_job],
        achievements_list[single_job],
        salary_list[single_job],
        equity_list[single_job]
    ]
    for value_index in range(len(total_columns)):
        total_values[value_index].append(single_values[value_index])
final_result = {}
for column_index in range(len(total_columns)):
    final_result.update({
        total_columns[column_index]: total_values[column_index]
    })
final_result = PD.DataFrame(final_result)

print("*****Total count {count}*****".format(count=count))
file_name = "jobs.csv"
print("*****Save the result to file {file}*****".format(file=file_name))
final_result.to_csv(file_name)


*****Processing job Senior Software Engineer (LavaMoat/Security Lab)*****
Type of position: Full-time
Job location: GLOBAL - Remote
Job description: https://consensys.net/open-roles/gh_jid?gh_jid=4788493
Working type: Others
Contract: talentre
Achievements: Null
Base salary: Not Posted
Equity: Not Posted
label: JavaScript,React Native,Blockchain,Quality Assurance,Mobile,Mobile Programming
*****Processing job Business Development Associate (San Francisco Bay Area - Silicon Valley)*****
Type of position: Full-time
Job location: San Francisco Bay Area, CA / Remote
Job description: https://jobs.lever.co/certik/8a3505fe-36a1-48a5-b3f7-b021da58582c
Working type: Others
Contract: talentre
Achievements: Null
Base salary: Not Posted
Equity: Not Posted
label: Blockchain,Software as a Service,Marketing,Product Roadmap,Business Development,SDR
*****Processing job Backend Engineer, On-Chain Data*****
Type of position: Full-time
Job location: Remote - US
Job description: https://boards.greenhouse.io

# 采用静态捕获的方法(已优化)

In [80]:
# make the bs soup into a list
with open('jobs.html', 'r', encoding='utf-8') as f:
 	soup_list = BeautifulSoup(f.read(), 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")

In [134]:
# get the initializd infomation
tag_list = []

job_title_list = []
type_list = {}
industry_list = {}
contract_list = {}
location_list = {}
equity_list = {}
achievements_list = {}
# loop = 0

for tag in soup_list.find_all('div',class_="job-list-job"):
        # print(tag)
        job_title_list.append(tag.find('a', target = "_blank").text)
        type_list[tag.find('a', target = "_blank").text] = 'Full-time'
        industry_list[tag.find('a', target = "_blank").text] = ['Others']
        contract_list[tag.find('a', target = "_blank").text] = 'talentre'
        # location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-badge-text").text if (not tag.find('span', class_ = "job-list-badge-text").text[0:6] == "Posted") else "Not Posted"
        location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-company-meta-item").text
        equity_list[tag.find('a', target = "_blank").text] = "Not Posted"
        achievements_list
        # loop += 1
        # if(loop == 10):
        #     break
        # for tag_ in tag.find_all('span',attrs={ 'data-toggle': 'tooltip'}):
        #     if('$' in tag_.text):
        #         tmp = tag_.text.split('$')[1].replace(',','').replace(')','')


In [135]:
# adjust the type_list
with open('jobs_internal.html', 'r', encoding='utf-8') as f:
 	soup_list_internal = BeautifulSoup(f.read(), 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")

In [136]:
for tag in soup_list_internal.find_all('div',class_="job-list-job"):
    type_list[tag.find('a', target = "_blank").text] = 'Internship'

In [98]:
# adjust the industry_list
industry = ['Engineer', 'Marketing']

In [137]:
# show all the results
count = 0
df = PD.DataFrame(columns=['Job title', 'Type of position', 'Job Location', 'Job description', 'Working type', 'Contact', 'Achievements', 'Base salary', 'Equity'])
for single_job in job_title_list:
    count += 1
    print("*****Processing job {job}*****".format(job=single_job))
    print("Type of position: " + type_list[single_job])
    print("Job location: " + location_list[single_job])
    print("Working type: " + (",".join(str(x) for x in industry_list[single_job])))
    print("Contract: " + contract_list[single_job])
    print("Achievements: " + achievements_list[single_job])
    print("Equity: " + equity_list[single_job])
    print("Achievements: " + achievements_list[single_job])
print("*****Total count {count}*****".format(count=count))

*****Processing job Business Operations Intern*****
Type of position: Internship
Job location: New York City Metropolitan Area, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Digital Marketing Assistant (Intern)*****
Type of position: Internship
Job location: New York, NY, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Marketing Assistant (Intern)*****
Type of position: Internship
Job location: New York, NY, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Platform Engineering Intern*****
Type of position: Internship
Job location: New York, NY  / Seattle, WA / San Francisco Bay Area, CA / Remote
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Internship Spring/Summer 2023 - Software Engineer, Tools*****
Type of position: Internship
Job location: New York, NY  / Seattle, WA / San Francisco Bay Area, CA / Remote
Working type: Others
Contract: talentre
Equi